<a href="https://colab.research.google.com/github/Cthompsonlbi/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-17 02:03:02--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.68MB/s    in 0.6s    

2022-02-17 02:03:04 (1.68 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
#Create a Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
#Connect to database and pull data from VinesTable
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://dataviz.cmfewojoukh8.us-east-1.rds.amazonaws.com:5432/Tools") \
    .option("dbtable", "vine_table") \
    .option("user", "postgres") \
    .option("password", "UCFRHPMyC21!?") \
    .option("driver", "org.postgresql.Driver") \
    .load()


In [7]:
#Check the Schema to make sure types are correct
df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [8]:
#Just showing some of the data pulled from the database to become familiar with the data
df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2UM5QMHBHC90Q|          5|            0|          0|   N|                Y|
|  RF0D1LEIF6L7|          4|            0|          0|   N|                Y|
|  RM6YKIWQVNSY|          1|            6|          6|   N|                Y|
|R1RL3L68ASPS36|          4|            0|          0|   N|                Y|
|R1U4XFBFAG34CY|          5|            0|          0|   N|                Y|
|R3KFIK8P0I91PL|          5|            0|          0|   N|                Y|
| RENOAY76PPK1O|          5|            0|          0|   N|                Y|
| RINV884I0NL5V|          1|            0|          0|   N|                Y|
| R5KJH6CXZH2PX|          5|            0|          0|   N|                Y|
| RO69JF6QWD0W1|          5|            0|          0|   N|     

In [9]:
#Just a count to ensure all data is present
df.count()

1741100

In [ ]:
#df.write.csv("Desktop/UCF/git/Amazon_Vine_Analysis/data/data.csv")

In [ ]:
#from google.colab import drive
#drive.mount('drive')

#df.write.csv('/content/drive/My Drive/filename.csv')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
#df.write.csv('/content/drive/My Drive/filename.csv')

In [ ]:
#TotalValue20df = Totaldf.where(Totaldf.total_votes >= '20').show()

In [ ]:
#TotalValue20df.printSchema()


In [10]:
#Don't need all columns, just selecting the columns required for the deliverables in the dataframe below
rev_df = df.select(["star_rating","helpful_votes","total_votes","vine","verified_purchase"])
rev_df.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
|          1|            6|          6|   N|                Y|
|          4|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          1|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          4|            1|          1|   N|                Y|
|          5|            0|          0|   N|                Y|
|          2|            0|          0|   N|           

In [11]:
#Just filters the data returning values of 20 or greater from the "total votes" column
rev20_df = rev_df.filter(rev_df["total_votes"]>= 20)
rev20_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           20|         23|   N|                Y|
|          5|           46|         51|   N|                Y|
|          4|           43|         43|   N|                Y|
|          5|           10|         22|   N|                Y|
|          5|           35|         36|   N|                Y|
|          1|           39|         45|   N|                N|
|          1|           21|         22|   N|                N|
|          5|            5|         21|   N|                Y|
|          1|            1|         22|   N|                Y|
|          5|           23|         24|   N|                N|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [12]:
#from the total votes column that had greater than 20 votes, it calculates those that have more favorable votes(helpful_votes
rev50_df = rev20_df.filter(rev20_df["helpful_votes"]/rev20_df["total_votes"] >= 0.50)
rev50_df.show(5)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           20|         23|   N|                Y|
|          5|           46|         51|   N|                Y|
|          4|           43|         43|   N|                Y|
|          5|           35|         36|   N|                Y|
|          1|           39|         45|   N|                N|
+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [13]:
#Filters the number of people who posted favorable reviews that had paid vine accounts
paid_rev_df = rev50_df.filter(rev50_df["vine"]== "Y")
paid_rev_df.show(5)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           38|         41|   Y|                N|
|          5|          175|        181|   Y|                N|
|          4|           63|         70|   Y|                N|
|          2|           36|         40|   Y|                N|
|          5|           19|         24|   Y|                N|
+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [14]:
#Summary Screen shot
paid_rev_df.describe().show()

+-------+------------------+-----------------+------------------+----+-----------------+
|summary|       star_rating|    helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+-----------------+------------------+----+-----------------+
|  count|               285|              285|               285| 285|              285|
|   mean| 4.326315789473684|48.12280701754386| 53.89122807017544|null|             null|
| stddev|0.9580140689279564|44.39073869889702|46.634087607753834|null|             null|
|    min|                 1|               12|                20|   Y|                N|
|    max|                 5|              340|               352|   Y|                Y|
+-------+------------------+-----------------+------------------+----+-----------------+



In [15]:
#Filters the number of people who posted favorable reviews that had unpaid vine accounts
unpaid_rev_df = rev50_df.filter(rev50_df["vine"] == "N")
unpaid_rev_df.show(5)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           20|         23|   N|                Y|
|          5|           46|         51|   N|                Y|
|          4|           43|         43|   N|                Y|
|          5|           35|         36|   N|                Y|
|          1|           39|         45|   N|                N|
+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [16]:
#Summary Screen shot
unpaid_rev_df.describe().show()

+-------+------------------+-----------------+-----------------+-----+-----------------+
|summary|       star_rating|    helpful_votes|      total_votes| vine|verified_purchase|
+-------+------------------+-----------------+-----------------+-----+-----------------+
|  count|             31545|            31545|            31545|31545|            31545|
|   mean| 3.683087652559835|47.51881439213822|51.85953399904898| null|             null|
| stddev|1.5261357705051182|84.07126117455323|87.94593778112954| null|             null|
|    min|                 1|               10|               20|    N|                N|
|    max|                 5|             4760|             4858|    N|                Y|
+-------+------------------+-----------------+-----------------+-----+-----------------+



In [17]:
#Number of 5 star ratings by paid vines
paid_5star= paid_rev_df[paid_rev_df["star_rating"]==5].count()
paid_5star

163

In [18]:
#Number of paind vines
paid_rev = paid_rev_df.count()
paid_rev

285

In [19]:
# Percentage of votes that are 5 star by paid vines
percent_paid_5star = paid_5star/paid_rev * 100
percent_paid_5star

57.19298245614035

In [20]:
#Number of 5 star ratings by unpaid vines
unpaid_5star = unpaid_rev_df[unpaid_rev_df['star_rating']==5].count()
unpaid_5star

14614

In [21]:
#Number of unpaid vines
unpaid_rev = unpaid_rev_df.count()
unpaid_rev

31545

In [22]:
# Percentage of votes that are 5 star by unpaid vines
percent_unpaid_5star = unpaid_5star/unpaid_rev * 100
percent_unpaid_5star

46.32746869551435